In [ ]:
!pip install gdown

In [ ]:
!pip install visualkeras

In [ ]:
import os
import glob
import gdown
import zipfile
import numpy as np
import shutil
import random
import torch
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import visualkeras
from tensorflow import keras
from tensorflow.keras import layers, callbacks
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications.resnet_v2 import preprocess_input
from tensorflow import data
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

In [ ]:
url = 'https://drive.google.com/uc?id=12wRN1bxPxJaRGQF5y2plz6ngZGZrLv39' 
path = './' 
gdown.download(url, path, quiet=False)

In [ ]:
z = zipfile.ZipFile('./WeatherImageRecognition.zip')
z.extractall()

In [ ]:
folder = './WeatherImageRecognition'

def readFilesAndLabels(folder):
    classes = os.listdir(folder)
    classes.sort()
    filenames = glob.glob(folder + '/*/*')
    print('Folder:', folder, 'Nb files :', len(filenames))
    labels = [classes.index(name.split('/')[-2]) for name in filenames]
    
    return filenames, labels, classes

filenames, labels, class_names = readFilesAndLabels(folder)

In [ ]:
root_dir = './WeatherImageRecognition'
dest_dir = './'
classes_dir = class_names

val_ratio = 0.1
test_ratio = 0.1

for cls in classes_dir:
    os.makedirs(dest_dir + 'train/' + cls)
    os.makedirs(dest_dir + 'val/'  + cls)
    os.makedirs(dest_dir + 'test/' + cls)

    src = root_dir + '/' + cls

    allFileNames = os.listdir(src)
    np.random.shuffle(allFileNames)
    train_FileNames, val_FileNames, test_FileNames = np.split(np.array(allFileNames),
                                                            [int(len(allFileNames)* (1 - (val_ratio + test_ratio))), 
                                                            int(len(allFileNames)* (1 - test_ratio))])

    train_FileNames = [src + '/' + name for name in train_FileNames.tolist()]
    val_FileNames   = [src + '/' + name for name in val_FileNames.tolist()]
    test_FileNames  = [src + '/' + name for name in test_FileNames.tolist()]

    print('Total images: ', len(allFileNames))
    print('Training: ', len(train_FileNames))
    print('Validation: ', len(val_FileNames))
    print('Testing: ', len(test_FileNames))
    
       # Copy-pasting images
    for name in train_FileNames:
        shutil.copy(name, dest_dir + 'train/' + cls)

    for name in val_FileNames:
        shutil.copy(name, dest_dir + 'val/'   + cls)

    for name in test_FileNames:
        shutil.copy(name, dest_dir + 'test/'  + cls)

In [ ]:
num_channels = 3
img_height = 128
img_width = 128
img_size = [img_height, img_width]

def read_image(filename, label):
    raw = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(raw, channels=num_channels)
    image = tf.image.resize(image, img_size)
    
    return image, label

In [ ]:
data_augmentation_1 = keras.Sequential([
    layers.Rescaling(scale=1.0/255),
    layers.RandomContrast(0.3),
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.1),
])

rescale_only = layers.Rescaling(scale=1.0/255)

In [ ]:
AUTOTUNE = data.experimental.AUTOTUNE

def configure_dataset(filenames, labels, augmentation=False, train=False):
    ds = tf.data.Dataset.from_tensor_slices((filenames, labels))
    ds = ds.map(read_image, num_parallel_calls=AUTOTUNE)
    
    if augmentation:
        ds = ds.repeat(3)
        ds = ds.map(lambda x,y: (data_augmentation_1(x, training=True), y), num_parallel_calls=AUTOTUNE)
    else:
        ds = ds.map(lambda x,y: (rescale_only(x), y), num_parallel_calls=AUTOTUNE)
        
    if train:
        ds = ds.shuffle(buffer_size=1000)
    ds = ds.cache()
    ds = ds.batch(batch_size=32)
    ds = ds.prefetch(AUTOTUNE)
    
    return ds

In [ ]:
filenames_train, labels_train, class_names = readFilesAndLabels('./train')
ds_train = configure_dataset(filenames_train, labels_train, augmentation=True, train=True)

filenames_valid, labels_valid, class_names = readFilesAndLabels('./val')
ds_valid = configure_dataset(filenames_valid, labels_valid)

filenames_test, labels_test, class_names = readFilesAndLabels('./test')
ds_test  = configure_dataset(filenames_test, labels_test)

In [ ]:
def show_img(imgs, labels, class_name):
    fig = plt.figure(figsize=(8, 8))
    columns = 4
    rows = 5
    for i in range(1, columns*rows +1):
        img = imgs[i]
        if num_channels==1:
            img = tf.squeeze(img)
        fig.add_subplot(rows, columns, i)
        plt.imshow(img)
        plt.title(class_name[labels[i]])
        plt.axis('off')
    plt.show()
    
for x in ds_test.take(1):
    imgs = x[0]
    labels = x[1]
    show_img(imgs, labels, class_names)

In [ ]:
model = keras.Sequential([
    # Input
    layers.InputLayer(input_shape=(img_height, img_width, num_channels)),
    
    # Block 1
    layers.BatchNormalization(),
    layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding='same', kernel_regularizer=l2()),
    layers.MaxPool2D(),
    
    # Block 2
    layers.BatchNormalization(),
    layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='same', kernel_regularizer=l2()),
    layers.MaxPool2D(),
    
    # Block 3
    layers.BatchNormalization(),
    layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='same', kernel_regularizer=l2()),
    layers.MaxPool2D(),
    
    # Block 4
    layers.BatchNormalization(),
    layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='same', kernel_regularizer=l2()),
    layers.MaxPool2D(),
    
    # Block 5
    layers.BatchNormalization(),
    layers.Conv2D(filters=512, kernel_size=3, activation='relu', padding='same', kernel_regularizer=l2()),
    layers.MaxPool2D(),
    
    # Head
    layers.BatchNormalization(),
    layers.Flatten(),
    layers.Dense(50, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(25, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(len(class_names), activation='softmax')
])

In [ ]:
model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.001, momentum=0.9, nesterov=True),
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'],
             )

In [ ]:
keras.utils.plot_model(model, "weather.png", show_shapes=True)

In [ ]:
visualkeras.layered_view(model, to_file='model1.png',legend=True)

In [ ]:
model.summary()

In [ ]:
os.makedirs('./checkpoint_path')
os.makedirs('./saved_model')

In [ ]:
EarlyStop = callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0.01, patience=15, verbose=1, restore_best_weights=True)

In [ ]:
checkpoint_path = './checkpoint_path'

checkpoint_dir = os.path.dirname(checkpoint_path)
CheckPoints = callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                              save_weights_only=True,
                                              verbose=1)

In [ ]:
training = 1
validation_freq = 2

if training == 1:
    epoch_max = 150
    history = model.fit(ds_train,
                        epochs=epoch_max,
                        validation_data=ds_valid,
                        validation_freq=validation_freq,
                        verbose=1,
                        callbacks=[CheckPoints, EarlyStop]
                       )
    
    model.save('./saved_model/model_trained')
    model.save_weights('./saved_model/model_trained_weight')
    
    train_acc = history.history['accuracy']
    train_loss = history.history['loss']
    val_acc = history.history['val_accuracy']
    val_loss = history.history['val_loss']
    
else:
    model = tf.keras.models.load_model('saved_model/model_trained')

In [ ]:
def display_plots(train_acc, valid_acc, train_loss, valid_loss, validation_freq):
    
    # accuracy plots
    x_valid = [(x+1)*validation_freq for x in range(0,len(valid_acc))]
    plt.figure(figsize=(12,9))
    plt.subplot(2,1,1)
    plt.plot(train_acc, color='green', linestyle='-', label='train_accuracy')
    plt.plot(x_valid, valid_acc, color='blue', linestyle='-', label='validation_accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    
    # loss plots
    plt.subplot(2,1,2)
    plt.plot(train_loss, color='orange', linestyle='-', label='train_loss')
    plt.plot(x_valid, valid_loss, color='red', linestyle='-', label='validation_loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.show()

In [ ]:
if training == 1:
    display_plots(train_acc, val_acc, train_loss, val_loss, validation_freq)

In [ ]:
y_raw = np.argmax(model.predict(ds_test), axis=1)
y_test = tf.concat([y for x,y in ds_test],axis=0).numpy()

In [ ]:
classification_report(y_test, y_raw)

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(confusion_matrix(y_test,y_raw),annot=True)

In [ ]:
data_augmentation = keras.Sequential([
    layers.RandomContrast(0.3),
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.1),
])

In [ ]:
img_height = 100
img_width = 100
img_size = [img_height, img_width]

def configure_dataset_2(filenames,labels,augmentation=False,train=False):
    ds = tf.data.Dataset.from_tensor_slices((filenames,labels)) 
    ds = ds.map(read_image, num_parallel_calls=AUTOTUNE)
    ds = ds.map(lambda x,y : (preprocess_input(x),y), num_parallel_calls=AUTOTUNE)

    if augmentation:
        ds = ds.repeat(3)
        ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y),num_parallel_calls=AUTOTUNE)
        
    if train:    
        ds = ds.shuffle(buffer_size=1000)
        
    ds = ds.cache()    
    ds = ds.batch(batch_size=32)
    ds = ds.prefetch(AUTOTUNE) 
    
    return ds

In [ ]:
filenames_train, labels_train, class_names = readFilesAndLabels('./train')
ds_train_2 = configure_dataset_2(filenames_train, labels_train, augmentation=True, train=True)

filenames_valid, labels_valid, class_names = readFilesAndLabels('./val')
ds_valid_2 = configure_dataset_2(filenames_valid, labels_valid)

filenames_test, labels_test, class_names = readFilesAndLabels('./test')
ds_test_2  = configure_dataset_2(filenames_test, labels_test)

In [ ]:
resnet50v2 = keras.applications.resnet_v2.ResNet50V2(input_shape=img_size+[num_channels], include_top=False, weights='imagenet')

for layer in resnet50v2.layers:
    layer.trainable = False

In [ ]:
resnet50v2.summary()

In [ ]:
x = layers.GlobalAvgPool2D()(resnet50v2.output)

x = layers.BatchNormalization()(x)
x = layers.Dense(50, activation='relu')(x)
x = layers.Dropout(0.5)(x)

x = layers.BatchNormalization()(x)
x = layers.Dense(25, activation='relu')(x)
x = layers.Dropout(0.3)(x)

x = layers.BatchNormalization()(x)
x = layers.Dense(len(class_names), activation='softmax')(x)

model = keras.Model(inputs=resnet50v2.input, outputs=x)

In [ ]:
initial_learning_rate=0.001

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=initial_learning_rate),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
keras.utils.plot_model(model, "weather2.png", show_shapes=True)

In [ ]:
visualkeras.layered_view(model, scale_xy=10, legend=True)

In [ ]:
model.summary()

In [ ]:
os.makedirs('./checkpoint_path2')
os.makedirs('./saved_model2')

In [ ]:
EarlyStop = callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0.01, patience=15, verbose=1, restore_best_weights=True)

In [ ]:
checkpoint_path = './checkpoint_path2'

checkpoint_dir = os.path.dirname(checkpoint_path)
CheckPoints = callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [ ]:
training = 1
validation_freq = 2

if training == 1:
    epoch_max = 150
    history = model.fit(ds_train_2,
                        epochs=epoch_max,
                        validation_data=ds_valid_2,
                        validation_freq=validation_freq,
                        verbose=1,
                        callbacks=[CheckPoints, EarlyStop]
                       )
    
    model.save('./saved_model2/model_trained')
    model.save_weights('./saved_model2/model_trained_weight')
    
    train_acc = history.history['accuracy']
    train_loss = history.history['loss']
    val_acc = history.history['val_accuracy']
    val_loss = history.history['val_loss']
    
else:
    model = keras.models.load_model('saved_model2/model_trained')

In [ ]:
display_plots(train_acc, val_acc, train_loss, val_loss, validation_freq)

In [ ]:
y_raw = np.argmax(model.predict(ds_test_2), axis=1)
y_test = tf.concat([y for x,y in ds_test_2],axis=0).numpy()

In [ ]:
classification_report(y_test, y_raw)

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(confusion_matrix(y_test,y_raw),annot=True)

In [ ]:
layerNo = 152

for i, layer in enumerate(resnet50v2.layers):
    layer.trainable = (i >= layerNo)
    print('i: ', i, 'name: ', layer.name, 'trainable: ', layer.trainable)

In [ ]:
fine_tune_lr = initial_learning_rate 
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=fine_tune_lr),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
training = 1
validation_freq = 2

if training == 1:
    epoch_max = 150
    history = model.fit(ds_train_2,
                        epochs=epoch_max,
                        validation_data=ds_valid_2,
                        validation_freq=validation_freq,
                        verbose=1,
                        callbacks=[CheckPoints, EarlyStop]
                       )
    
    model.save('./saved_model2/model_trained')
    model.save_weights('./saved_model2/model_trained_weight')
    
    train_acc = history.history['accuracy']
    train_loss = history.history['loss']
    val_acc = history.history['val_accuracy']
    val_loss = history.history['val_loss']
    
else:
    model = keras.models.load_model('saved_model2/model_trained')

In [ ]:
display_plots(train_acc, val_acc, train_loss, val_loss, validation_freq)

In [ ]:
y_raw = np.argmax(model.predict(ds_test_2), axis=1)
y_test = tf.concat([y for x,y in ds_test_2],axis=0).numpy()

In [ ]:
classification_report(y_test, y_raw)

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(confusion_matrix(y_test,y_raw),annot=True)

In [ ]:
last_epoch = history.epoch[-1]
final_epoch = last_epoch + 40
validation_freq = 1

history_fine = model.fit(ds_train_2,
                         initial_epoch=last_epoch,
                         epochs=final_epoch,
                         validation_data=ds_valid_2,
                         validation_freq=validation_freq,
                         verbose=1,
                         callbacks=[CheckPoints, EarlyStop]
                         )

model.save('./saved_model2/model_trained')
model.save_weights('./saved_model2/model_trained_weight')
    
train_acc = history_fine.history['accuracy']
train_loss = history_fine.history['loss']
val_acc = history_fine.history['val_accuracy']
val_loss = history_fine.history['val_loss']

In [ ]:
y_raw = np.argmax(model.predict(ds_test_2), axis=1)
y_test = tf.concat([y for x,y in ds_test_2],axis=0).numpy()

In [ ]:
classification_report(y_test, y_raw)

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(confusion_matrix(y_test,y_raw),annot=True)

In [ ]:
fine_tune_lr_2 = fine_tune_lr / 10
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=fine_tune_lr),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
last_epoch = history.epoch[-1]
final_epoch = last_epoch + 40
validation_freq = 1

history_fine = model.fit(ds_train_2,
                         initial_epoch=last_epoch,
                         epochs=final_epoch,
                         validation_data=ds_valid_2,
                         validation_freq=validation_freq,
                         verbose=1,
                         callbacks=[CheckPoints, EarlyStop]
                         )

model.save('./saved_model2/model_trained')
model.save_weights('./saved_model2/model_trained_weight')
    
train_acc = history_fine.history['accuracy']
train_loss = history_fine.history['loss']
val_acc = history_fine.history['val_accuracy']
val_loss = history_fine.history['val_loss']

In [ ]:
y_raw = np.argmax(model.predict(ds_test_2), axis=1)
y_test = tf.concat([y for x,y in ds_test_2],axis=0).numpy()

In [ ]:
classification_report(y_test, y_raw)

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(confusion_matrix(y_test,y_raw),annot=True)